In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import random
import math
import re

class Tache:
    def __init__(self, nom, patient, competences, duree, docteur=None):
        self.nom = nom
        self.patient = patient
        self.competences = competences
        self.duree = duree
        self.docteur = docteur

class GestionnaireTaches:
    def __init__(self, docteurs):
        self.taches = []
        self.docteurs = docteurs
        self.amis = {}
        self.ennemis = {}

    def ajouter_amis(self, personne, amis):
        self.amis[personne] = amis

    def ajouter_ennemis(self, personne, ennemis):
        self.ennemis[personne] = ennemis

    def afficher_amis(self):
        if not self.amis:
            print("Aucune relation d'amitié définie.")
            return
        for personne, liste_amis in self.amis.items():
            amis_formatte = ", ".join(liste_amis)
            print(f"{personne} a pour amis : {amis_formatte}")

    def afficher_ennemis(self):
        if not self.ennemis:
            print("Aucune relation d'ennemi définie.")
            return
        for personne, liste_ennemis in self.ennemis.items():
            ennemis_formatte = ", ".join(liste_ennemis)
            print(f"{personne} a pour ennemis : {ennemis_formatte}")

    def charger_depuis_csv(self, chemin_fichier):
        data = pd.read_csv(chemin_fichier)
        for _, row in data.iterrows():
            for op in row.index[1:]:
                competences_str = row[op].replace(" et ", ", ").replace("Aucune opération", "")
                competences = competences_str.split(", ") if competences_str else []
                if competences:
                    tache = Tache(op, row.iloc[0], competences, self.calculer_duree(competences_str))
                    self.assigner_docteur(tache)
                    self.taches.append(tache)

    def assigner_docteur(self, tache):
        docteurs_potentiels = self.docteurs[:]
        amis = self.amis.get(tache.patient, [])
        docteurs_amis = [docteur for docteur in docteurs_potentiels if docteur in amis]
        if docteurs_amis:
            tache.docteur = random.choice(docteurs_amis)
        else:
            ennemis = self.ennemis.get(tache.patient, [])
            docteurs_non_ennemis = [docteur for docteur in docteurs_potentiels if docteur not in ennemis]
            tache.docteur = random.choice(docteurs_non_ennemis) if docteurs_non_ennemis else random.choice(docteurs_potentiels)

    @staticmethod
    def calculer_duree(competences_str):
        return sum(int(m.group(2)) if m.group(2) else 1 for m in re.finditer(r'C\d+(x(\d+))?', competences_str))

    def duree_par_patient(self):
        duree_totale_par_patient = {}
        for tache in self.taches:
            duree_totale_par_patient.setdefault(tache.patient, 0)
            duree_totale_par_patient[tache.patient] += tache.duree
        return duree_totale_par_patient

    def afficher_taches_et_docteurs(self):
        if not self.taches:
            print("Aucune tâche assignée.")
            return
        for tache in self.taches:
            print(f"Tâche: {tache.nom}, Patient: {tache.patient}, Docteur assigné: {tache.docteur}, Durée: {tache.duree} minutes")

class RecuitSimule:
    def __init__(self, taches):
        self.taches = taches

    def cout_ordre(self, ordre):
        cout_total = 0
        temps_cumule_par_patient = {}
        for tache in ordre:
            cout_total += tache.duree
            temps_cumule_par_patient.setdefault(tache.patient, 0)
            temps_cumule_par_patient[tache.patient] += tache.duree
        cout_total += max(temps_cumule_par_patient.values()) - min(temps_cumule_par_patient.values())
        return cout_total

    def generer_ordre_voisin(self, ordre):
        voisin = ordre[:]
        index1, index2 = random.sample(range(len(ordre)), 2)
        voisin[index1], voisin[index2] = voisin[index2], voisin[index1]
        return voisin

    def optimiser(self, T0, alpha, nb_iterations):
        T = T0
        meilleur_ordre = self.taches[:]
        meilleur_cout = self.cout_ordre(meilleur_ordre)
        for _ in range(nb_iterations):
            ordre_voisin = self.generer_ordre_voisin(meilleur_ordre)
            cout_voisin = self.cout_ordre(ordre_voisin)
            delta_cout = cout_voisin - meilleur_cout
            if delta_cout < 0 or random.random() < math.exp(-delta_cout / T):
                meilleur_ordre = ordre_voisin
                meilleur_cout = cout_voisin
            T *= alpha
        return meilleur_ordre, meilleur_cout

# Exemple d'utilisation
docteurs = ["Dr. Smith", "Dr. Johnson", "Dr. Williams"]
gestionnaire = GestionnaireTaches(docteurs)

# Définir des amis et des ennemis si nécessaire
gestionnaire.ajouter_amis("Dr. Smith", ["Dr. Johnson", "Dr. Williams"])
gestionnaire.ajouter_ennemis("Patient Y", ["Dr. Johnson"])

# Afficher les amis et les ennemis
gestionnaire.afficher_amis()
gestionnaire.afficher_ennemis()

# Charger les tâches à partir d'un fichier CSV (ajustez le chemin selon vos besoins)
gestionnaire.charger_depuis_csv('/kaggle/input/recuit/Fil rouge M2 - Table_competences.csv')

# Afficher les tâches assignées et les docteurs
gestionnaire.afficher_taches_et_docteurs()

# Calculer et afficher la durée totale des tâches par patient
duree_totale_par_patient = gestionnaire.duree_par_patient()
print("\nDurée totale des tâches par patient :")
for patient, duree in duree_totale_par_patient.items():
    print(f"Patient {patient} : Durée totale des tâches = {duree} minutes")

# Initialiser et exécuter l'optimisation par recuit simulé
recuit = RecuitSimule(gestionnaire.taches)
T0 = 100.0
alpha = 0.95
nb_iterations = 1000
ordre_optimal, cout_optimal = recuit.optimiser(T0, alpha, nb_iterations)

# Afficher le coût optimal et l'ordre optimal des tâches
print("\nCoût optimal:", cout_optimal)
print("Ordre optimal des tâches :")
for tache in ordre_optimal:
    print(f"Tâche: {tache.nom}, Patient: {tache.patient}")


Dr. Smith a pour amis : Dr. Johnson, Dr. Williams
Patient Y a pour ennemis : Dr. Johnson
Tâche: Opération 1, Patient: Patient 1, Docteur assigné: Dr. Smith, Durée: 2 minutes
Tâche: Opération 2, Patient: Patient 1, Docteur assigné: Dr. Smith, Durée: 2 minutes
Tâche: Opération 3, Patient: Patient 1, Docteur assigné: Dr. Smith, Durée: 2 minutes
Tâche: Opération 4, Patient: Patient 1, Docteur assigné: Dr. Smith, Durée: 3 minutes
Tâche: Opération 5, Patient: Patient 1, Docteur assigné: Dr. Johnson, Durée: 4 minutes
Tâche: Opération 1, Patient: Patient 2, Docteur assigné: Dr. Johnson, Durée: 2 minutes
Tâche: Opération 2, Patient: Patient 2, Docteur assigné: Dr. Smith, Durée: 2 minutes
Tâche: Opération 3, Patient: Patient 2, Docteur assigné: Dr. Smith, Durée: 1 minutes
Tâche: Opération 1, Patient: Patient 3, Docteur assigné: Dr. Smith, Durée: 2 minutes
Tâche: Opération 2, Patient: Patient 3, Docteur assigné: Dr. Williams, Durée: 1 minutes
Tâche: Opération 1, Patient: Patient 4, Docteur assign